In [1]:
import time
# import pigpio

In [2]:
# pi1 = pigpio.pi()

In [3]:
red_pin = 5       # change these for your values
green_pin = 13
blue_pin = 26
rgb_pins = [red_pin, green_pin, blue_pin]

# pi1.set_mode(red_pin,pigpio.OUTPUT)     # I don't think this is strictly necessary, but it's 
# pi1.set_mode(green_pin,pigpio.OUTPUT)   # used in the documentation - seems to be best practices
# pi1.set_mode(blue_pin,pigpio.OUTPUT)

#### valid_int():

Checks to see if the input the user provided falls between 0 and 255.

In [4]:
def valid_int(s):
    try:
        x = int(s)
        if(x > 255 or x < 0):
            return False
        else:
            return True
    except ValueError:
        return False

#### user_color():

This function asks the user to provide a number between 0 and 255 for red, green, and blue. Pretty self explanatory.

This isn't the best way to interface with your LEDs, but this is just to demonstrate the idea. The final product - when we have a Flask web application running - will much cooler

In [5]:
def user_color():
    
    red_led = input("value for red led (0-255) -->")
    while valid_int(red_led) != True:
            print("invalid input for the red LEDs")
            red_led = input("please enter a value between 0 and 255 -->")
    green_led = input("value for green led (0-255) ")
    while valid_int(green_led) != True:
            print("invalid input for the green LEDs -->")
            green_led = input("please enter a value between 0 and 255 -->")
    blue_led = input("value for blue led (0-255) -->")
    while valid_int(blue_led) != True:
            print("invalid input for the blue LEDs")
            blue_led = input("please enter a value between 0 and 255 -->")
    rgb = [int(red_led), int(green_led), int(blue_led)]
    change_colors(rgb)

#### change_colors():

In user_color() we make a list containing the new values we'd like to use as the PWM dutycycles for our GPIO pins. This just contains a simple loop to go through and set these.

In [6]:
# def change_colors(rgb_vals):
#     for pin, new_duty in (rgb_pins, rgb_vals):
#         pi1.set_PWM_dutycycle(pin, new_duty)
def change_colors(rgb_vals):
    pass

Alright, now let's see if this works!

In [7]:
user_color()

value for red led (0-255) -->0
value for green led (0-255) 100
value for blue led (0-255) -->5000
invalid input for the blue LEDs
please enter a value between 0 and 255 -->-100
invalid input for the blue LEDs
please enter a value between 0 and 255 -->just some text
invalid input for the blue LEDs
please enter a value between 0 and 255 -->255


Everything looking good, it even ignores my deliberately bad input!

#### color_fade():

One problem I ran into while I was testing this, was that the LEDs would hurt your eyes if they transitioned quickly from very different colors - from vivid red to bright blue, for example. My solution was to add a small transition time during which the LEDs would fade from one color to the next.

The nice thing about this function is that we can use it to make effects with our LEDs later on. By playing with the time interval, we can slowly fade between colors if we want a more dynamic effect.

You can either try this out by calling it manually:
```python
color_fade([100,150,0],t=1.5)
```
or you can change the line in user_color() that calls change_colors()

In [8]:
def color_fade(rgb_vals, t=1.):
    '''
    Input:
        rgb: a list containing the desired new values for 
        the red, green, and blue LEDs.
        
        t: the time the transition to the new color should 
        take. Default is 1 second. 
    '''    
    
    
    '''
    These first few lines make a list containing the PWM dutycycles 
    for each GPIOpin.
    The try/except is in there because if we're running this script 
    without having run any other pigpio commands, the pigpio module
    will return the error 'GPIO is not in use for PWM,' because
    we haven't told pigpio to use the pins for PWM. We're asking it 
    to do something nonsensical - the except assigns the 'current'
    values to zero.
    I wish there was a snazzier way to do this, but looking at the 
    documentation, I don't think there is.
    '''
    try:   
        rgb_current = [pi1.get_PWM_dutycycle(i) for i in rgb_pins]

    except pigpio.error:
        rgb_current = [0,0,0]
    
    # makes a list containing the differences between the current
    # state of each pin and the desired state
    diff_list = [(v-c) for c,v in zip(rgb_current, rgb_vals)]
    
    # Get the smallest non-zero difference 
    min_diff = min([abs(i) if i != 0 else 1000 for i in diff_list])
    
    # Exit if the RGB values are already all the same
    if min_diff == 1000: return 
    
    # How much we're going to incriment each pin by per step
    step_sizes = [i/min_diff for i in diff_list]
    
    # A small pause between each loop, so the transition takes the 
    # right amount of time...
    time_inc = t/min_diff
    
    '''
    We go through and calculate how much each value should have changed
    (step-size for that value * number of loops). The pin with the 
    smallest non-zero adjustment will be have its value incrimented by
    1 each loop. We change the PWM for each pin, pause briefly, and 
    repeat. The entire process takes 't' seconds. 
    '''
    for i in range(min_diff + 1):
        for pin, val, stp in zip(rgb_pins, rgb_current, step_sizes):
            new_duty = val+int(stp*i)
            pi1.set_PWM_dutycycle(pin, new_duty)
        time.sleep(time_inc)
    # Done!

### Should be good to go! 
Email me if you have any problems: timothy.salazar@colorado.edu